# miBrain Plotting analysis

### Assuming XLSX DEGs are in "miBrain" directory, load the files:

In [ ]:
library(miBrain)
set_ht_opt(2)
dl = lapply(list.files("miBrain",
                       pattern="^[A-Za-z]+[.]xlsx$", 
                       full.names=TRUE), 
            deg_load)
names(dl) = sapply(dl, function(obj) { obj@metadata$comparison })
dl

### Calculate DEG heatmaps, volcano plots, DEG heatmaps:

In [ ]:
library(patchwork)
g=Reduce("+", lapply(dl, function(se) {
    ct = S4Vectors::metadata(se)$comparison
    column_title = paste0("Top DEGs in ", ct)
    dir.create(paste0("miBrain/byct/", ct), recursive=TRUE)
    se = se_tmm(se, log=TRUE)
    cd = as.data.frame(SummarizedExperiment::colData(se))
    mono = setdiff(1:nrow(cd), grep("miBrain", cd$group))
    cd$group = gsub("^mono ", "", cd$group)
    cd$group[mono] = paste0("monoculture ", cd$group[-grep("miBrain", cd$group)])
    se$group = cd$group
    se = se[,order(se$group)]
    H = gene_heatmap(se, column_title=gsub("[.][0-9]+$", "", column_title))
    #H = H + ComplexHeatmap::rowAnnotation(counts=ComplexHeatmap::anno_barplot(colSums(SummarizedExperiment::assays(se)$counts)))
    saveHeatmap(H, paste0("miBrain/byct/", ct, "/heatmaps_deg"))
    g = PlotPCA(se)
    saveGGplot(g, paste0("miBrain/byct/", ct, "/pca"), w=3, h=3)
    g = volcano(S4Vectors::metadata(se)$deg, title=paste0("DEGs in ", ct))
    saveGGplot(g, paste0("miBrain/byct/", ct, "/volcano"), w=5, h=5)
    return(g)
}))

g

### Parsing XLSX gene lists per cell type

In [ ]:
df = xl_parse_gene_list(system.file("extdata", "heatmap_genes.xlsx", package="miBrain"))
df

In [ ]:
sapply(split(df, df$go_name), function(gf) {
    ct = unique(gf$celltype)
    column_title=head(gf$go_name, 1)
    se = se_tmm(dl[[ct]], log=TRUE)
    cd = as.data.frame(SummarizedExperiment::colData(se))
    mono = setdiff(1:nrow(cd), grep("miBrain", cd$group))
    cd$group = gsub("^mono ", "", cd$group)
    cd$group[mono] = paste0("monoculture ", cd$group[-grep("miBrain", cd$group)])
    se$group = cd$group
    se = se[,order(se$group)]
    H = gene_heatmap(se, gene=gf$gene, column_title=gsub("[.][0-9]+$", "", column_title))
    saveHeatmap(H, paste0("miBrain/byct/", ct, "/", column_title))
})